<a href="https://colab.research.google.com/github/jesst0x/image-segmentation-in-dark/blob/main/notebook/fine_tuned_mask_rcnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

FOLDERNAME = 'AI/'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# Now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

Mounted at /content/drive


In [2]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2659  100  2659    0     0  28094      0 --:--:-- --:--:-- --:--:-- 28287
OK
45 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: http://packages.cloud.google.com/apt/dists/gcsfuse-bionic/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 10.5 MB of archives.
After this operation, 0 B of additional disk space will be used.
Selecting previously unselected package gcsfuse.
(Reading database ... 121918 files and directories currently installed.)
Preparing to unpack .../gcsfuse_2.1.0_amd64.deb ...
Unpacking gcsfuse (2.1.0) ...
Setting up gcsfuse (2.1.0) ...


In [3]:
### Connect to gs bucket
from google.colab import auth
auth.authenticate_user()

In [4]:
# Loading data from gs bucket
!mkdir coco
!gcsfuse cs231n_lls coco
!gcsfuse --implicit-dirs cs231n_lls coco

{"timestamp":{"seconds":1716773273,"nanos":689614606},"severity":"INFO","message":"Start gcsfuse/2.1.0 (Go version go1.22.3) for app \"\" using mount point: /content/coco\n"}
{"timestamp":{"seconds":1716773273,"nanos":689818034},"severity":"INFO","message":"GCSFuse mount command flags: {\"AppName\":\"\",\"Foreground\":false,\"ConfigFile\":\"\",\"MountOptions\":{},\"DirMode\":493,\"FileMode\":420,\"Uid\":-1,\"Gid\":-1,\"ImplicitDirs\":false,\"OnlyDir\":\"\",\"RenameDirLimit\":0,\"IgnoreInterrupts\":false,\"CustomEndpoint\":null,\"BillingProject\":\"\",\"KeyFile\":\"\",\"TokenUrl\":\"\",\"ReuseTokenFromUrl\":true,\"EgressBandwidthLimitBytesPerSecond\":-1,\"OpRateLimitHz\":-1,\"SequentialReadSizeMb\":200,\"AnonymousAccess\":false,\"MaxRetrySleep\":30000000000,\"StatCacheCapacity\":20460,\"StatCacheTTL\":60000000000,\"TypeCacheTTL\":60000000000,\"HttpClientTimeout\":0,\"MaxRetryDuration\":-1000000000,\"RetryMultiplier\":2,\"LocalFileCache\":false,\"TempDir\":\"\",\"ClientProtocol\":\"http1

In [5]:
import os
# Get reference util library from pytorch.
os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/engine.py")
os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/utils.py")
os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_utils.py")
os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_eval.py")
os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/transforms.py")

0

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision.transforms.v2 as transforms
import torchvision.transforms.functional as F
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor, PILToTensor
from torchvision.utils import make_grid, draw_segmentation_masks
from torchvision.models.detection import maskrcnn_resnet50_fpn, MaskRCNN_ResNet50_FPN_Weights
from torchvision import models, datasets, tv_tensors
from torchvision.io import read_image
from pathlib import Path
from engine import train_one_epoch, evaluate
import utils

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [7]:
def show(imgs):
    if not isinstance(imgs, list):
        imgs = [imgs]
    fig, axs = plt.subplots(ncols=len(imgs), squeeze=False)
    for i, img in enumerate(imgs):
        img = img.detach()
        img = F.to_pil_image(img)
        axs[0, i].imshow(np.asarray(img))
        axs[0, i].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])

In [8]:
# train on the gpu, or on cpu if a gpu is not available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

###Loading Dataset

In [10]:
### Loading Training dataset
TRAINING_IMAGES_PATH = Path("/content/coco/coco/coco_train5000_augmented")
TRAINING_ANNOTATIONS_PATH = Path("/content/coco/coco/instances_train2017_subset5000.json")

train_dataset = datasets.CocoDetection(TRAINING_IMAGES_PATH, TRAINING_ANNOTATIONS_PATH, transform=ToTensor())
train_dataset = datasets.wrap_dataset_for_transforms_v2(train_dataset, target_keys=("all"))
l, t = train_dataset[0]

# Only take subset of 200 for quick experiment at colab
indices = torch.randperm(len(train_dataset)).tolist()
subset = torch.utils.data.Subset(train_dataset, indices[:200])

train_dataloader = torch.utils.data.DataLoader(
    subset,
    batch_size=4,
    shuffle=True,
    collate_fn=utils.collate_fn,
    num_workers=2,
)

loading annotations into memory...
Done (t=0.80s)
creating index...
index created!


In [ ]:
label, target = next(iter(train_dataloader))
# print(label[0])
print(target[0].keys())

dict_keys(['image_id', 'boxes', 'masks', 'labels', 'bbox', 'segmentation', 'area', 'iscrowd', 'category_id'])


In [ ]:
### Loading 500 Synthetic Low Light Validation Set
VAL500_AUGMENTED_IMAGES_PATH = Path("/content/coco/coco/coco_val500_augmented")
VAL500_ANNOTATIONS_PATH = Path("/content/drive/MyDrive/AI/coco/instances_val2017_subset500.json")

val_augmented_dataset = datasets.CocoDetection(VAL500_AUGMENTED_IMAGES_PATH, VAL500_ANNOTATIONS_PATH, transform=ToTensor())
val_augmented_dataset = datasets.wrap_dataset_for_transforms_v2(val_augmented_dataset, target_keys=("all"))
val_augmented_dataloader = torch.utils.data.DataLoader(
    val_augmented_dataset,
    batch_size=8,
    shuffle=True,
    collate_fn=utils.collate_fn,
)


loading annotations into memory...
Done (t=0.25s)
creating index...
index created!


In [ ]:
### Loading 500 Real Natural Light Validation Set from COCO
VAL500_IMAGES_PATH = Path("/content/coco/coco/val500")
VAL500_ANNOTATIONS_PATH = Path("/content/drive/MyDrive/AI/coco/instances_val2017_subset500.json")
val_original_dataset = datasets.CocoDetection(VAL500_IMAGES_PATH, VAL500_ANNOTATIONS_PATH, transform=ToTensor())
val_original_dataset = datasets.wrap_dataset_for_transforms_v2(val_original_dataset, target_keys=("all"))
val_original_dataloader = torch.utils.data.DataLoader(
    val_original_dataset,
    batch_size=8,
    shuffle=True,
    collate_fn=utils.collate_fn,
)


loading annotations into memory...
Done (t=0.04s)
creating index...
index created!


#Training

After loading pre-trained weight into Mask R-CNN model, fine tuning by continue training using synthetic low light images dataset, instead of natural light COCO dataset.

In [ ]:
weights = MaskRCNN_ResNet50_FPN_Weights.DEFAULT
finetune_model = maskrcnn_resnet50_fpn(weights=weights, progress=False)
finetune_model.to(device)

# Optimizer
params = [p for p in finetune_model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params, lr=0.00005)

# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer,
    step_size=3,
    gamma=0.1
)

# finetune_model.load_state_dict(torch.load('model_weights.pth'))

Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/hub/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth


In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    # Training
    train_one_epoch(finetune_model, optimizer, train_dataloader, device, epoch, print_freq=10)
    # Update the learning rate
    lr_scheduler.step()

# Save model weight for
torch.save(finetune_model.state_dict(), 'model_weights.pth')

Epoch: [0]  [ 0/50]  eta: 0:05:58  lr: 0.000001  loss: 2.1352 (2.1352)  loss_classifier: 0.6153 (0.6153)  loss_box_reg: 0.2389 (0.2389)  loss_mask: 0.4768 (0.4768)  loss_objectness: 0.7333 (0.7333)  loss_rpn_box_reg: 0.0710 (0.0710)  time: 7.1752  data: 2.8459  max mem: 6001
Epoch: [0]  [10/50]  eta: 0:01:16  lr: 0.000011  loss: 1.3989 (1.4501)  loss_classifier: 0.3985 (0.4081)  loss_box_reg: 0.2726 (0.2548)  loss_mask: 0.4170 (0.4113)  loss_objectness: 0.2683 (0.3093)  loss_rpn_box_reg: 0.0434 (0.0666)  time: 1.9166  data: 0.2714  max mem: 7347
Epoch: [0]  [20/50]  eta: 0:00:53  lr: 0.000021  loss: 1.3023 (1.4204)  loss_classifier: 0.3814 (0.4017)  loss_box_reg: 0.2726 (0.2608)  loss_mask: 0.3931 (0.3991)  loss_objectness: 0.1309 (0.2883)  loss_rpn_box_reg: 0.0434 (0.0705)  time: 1.4987  data: 0.0139  max mem: 7679
Epoch: [0]  [30/50]  eta: 0:00:33  lr: 0.000032  loss: 1.0735 (1.3325)  loss_classifier: 0.3617 (0.3694)  loss_box_reg: 0.2540 (0.2413)  loss_mask: 0.3738 (0.3963)  loss_ob

In [ ]:
evaluate(finetune_model, val_augmented_dataloader, device=device)
evaluate(finetune_model, val_original_dataloader, device=device)

Test:  [ 0/63]  eta: 0:06:58  model_time: 2.4631 (2.4631)  evaluator_time: 0.2425 (0.2425)  time: 6.6391  data: 3.9271  max mem: 12661
Test:  [62/63]  eta: 0:00:05  model_time: 1.5816 (1.5881)  evaluator_time: 0.2439 (0.3103)  time: 4.3942  data: 2.5636  max mem: 12668
Test: Total time: 0:05:20 (5.0825 s / it)
Averaged stats: model_time: 1.5816 (1.5881)  evaluator_time: 0.2439 (0.3103)
Accumulating evaluation results...
DONE (t=0.92s).
Accumulating evaluation results...
DONE (t=0.91s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.161
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.282
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.172
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.085
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.174
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.246
 Average Rec